In [1]:
import sys
from chart_studio import plotly
import plotly.graph_objs as go
import plotly.offline as pltlyoff
import numpy as np
import matplotlib.pyplot as plt

### Custom files ###
dir_GPPlotly = 'C:\\Users\\tln229\\Downloads\\Python Tests\\Constrained GP_2\\GP_2\\gp_plotly\\'
sys.path.append(dir_GPPlotly)

dir_gp_constr = 'C:\\Users\\tln229\\Downloads\\Python Tests\\Constrained GP_2\\GP_2\\gp_constr\\'
sys.path.append(dir_gp_constr)

from GPPlotly.plottingfunctions import PlotGP2d, add_traces_to_fig
from GPConstr.model import GPmodel, Constraint
from GPConstr.kern import kernel_RBF

Unable to determine R home: [WinError 2] The system cannot find the file specified


Loading constrained GP module from C:\Users\tln229\Downloads\Python Tests\Constrained GP_2\GP_2\gp_constr
Loading R wrapper...


R[write to console]: Loading required package: mvtnorm

R[write to console]: Loading required package: Matrix

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: gmm

R[write to console]: Loading required package: sandwich

R[write to console]: 
Attaching package: 'TruncatedNormal'


R[write to console]: The following objects are masked from 'package:tmvtnorm':

    dtmvnorm, dtmvt, ptmvnorm, ptmvt, rtmvnorm, rtmvt


R[write to console]: The following objects are masked from 'package:mvtnorm':

    pmvnorm, pmvt




Running R from rpy2: R version 4.2.2 (2022-10-31 ucrt)


In [2]:
# ---------- DATA ----------
def fun(x):
    return (np.arctan(20*x - 10) - np.arctan(-10))/3

# Design data - no noise
x_train = np.array([1/(n+2) for n in range(7)] ) + 0.1
y_train = fun(x_train)

# For plotting
x_test = np.linspace(0, 1, 500)
y_true = fun(x_test)

In [3]:
# Set up model
ker = kernel_RBF(variance = 0.5, lengthscale = [0.1])
model = GPmodel(kernel = ker, likelihood = 1E-6, mean = 0)

# Add the training data
model.X_training = x_train.reshape(-1, 1)
model.Y_training = y_train

In [4]:
# Plot unconstrained un-optimized GP
mean_unconstr, cov_unconstr = model.calc_posterior_unconstrained(x_test.reshape(-1, 1), full_cov = True)
mean_unconstr = np.array(mean_unconstr).flatten()
var_unconstr = np.diagonal(cov_unconstr)

num_samples = 5
samplepaths_unconstr = np.random.multivariate_normal(mean_unconstr, cov_unconstr, num_samples).T

fig_unconstr_1 = PlotGP2d(x_mean = x_test,
                        mean = mean_unconstr,
                        var = var_unconstr,
                        x_obs = model.X_training,
                        y_obs = model.Y_training, 
                        num_std = 1.2815,
                        x_true = x_test,
                        y_true = y_true,
                        samplepaths = samplepaths_unconstr,
                        title = 'Unconstrained GP',
                        xrange = [0, 1],
                        yrange = [-2, 2],
                        smoothing = True)

pltlyoff.iplot(fig_unconstr_1, filename='')

..Running calculation of K_w ... DONE - time: 0.000 seconds
..Running calculation of Cholesky factor for K_w ... DONE - time: 0.000 seconds
..Calculating f* | Y ... DONE - Total time: 0.007 seconds


In [5]:
# Helper functions for constraints
def constant_function(val):
    """ Return the constant function"""
    def fun(x):
        return np.array([val]*x.shape[0])
    return fun

In [6]:
# Define constraints for bounding the function and its derivative
constr_bounded = Constraint(LB = constant_function(0), UB = constant_function(1))
constr_deriv = Constraint(LB = constant_function(0), UB = constant_function(float('Inf')))

In [7]:
# Add only boundedness constraint to model
model.constr_bounded    = constr_bounded
model.constr_likelihood = 1E-6

In [8]:
# # Search for a suitable set of virtual observation locations where the constraint is imposed
# df, num_pts, pc_min = model.find_XV_subop(bounds = [(0.001, 1)], p_target = 0.99, i_range = [0], print_intermediate = True) # (Some intermediate calcs are stored in df)
# XV_bound = model.constr_bounded.Xv # (Extract the virtual observation locations for plotting)

# Note: Alternatively we can set these points manually, 
# e.g. model.constr_bounded.Xv = np.linspace(0, 1, 20).reshape(-1, 1)
model.constr_bounded.Xv = np.linspace(0, 1, 20).reshape(-1, 1)

In [9]:
# Print the model
print(model)

----- GP model ----- 
 mean = 0 
 likelihood = 1e-06 
 kernel: 
   type = RBF 
   input dim = 1 
   lenghtscale = [0.1] 
   variance = 0.5 
 constraint: 
   f [20] 
   constr_likelihood = 1e-06 
---------------------


In [10]:
# Plot model with boundedness constraint
mean, var, perc, mode, samples, times = model.calc_posterior_constrained(x_test.reshape(-1, 1), compute_mode = False, num_samples = 10000, save_samples = 5, algorithm = 'minimax_tilting', resample = False)

mean = np.array(mean).flatten()
p_lower = perc[0]
median = perc[1]
p_upper = perc[2]
p_label = '[p{}, p{}]'.format(10, 90)
samplepaths_Z = np.array(samples)

# Get GP figure
fig_bounded = PlotGP2d(x_mean = x_test,
                        mean = mean,
                        x_obs = model.X_training,
                        y_obs = model.Y_training, 
                        samplepaths =  samplepaths_Z,
                        x_true = x_test,
                        y_true = y_true,
                        p_lower = p_lower,
                        p_upper = p_upper,
                        p_label = p_label,
                        title = 'Boundedness constraint',
                        xrange = [0, 1],
                        yrange = [-1.7, 1.7],
                        smoothing = True)

# Add traces for bounds and virtual observation locations
trace_UB = go.Scatter(x = x_test, y = model.constr_bounded.UB(x_test), mode = 'lines', name = 'Upper bound', line = dict(color = ('rgb(0, 0, 0)'), shape = 'spline', width = 1))
trace_LB = go.Scatter(x = x_test, y = model.constr_bounded.LB(x_test), mode = 'lines', name = 'Lower bound', line = dict(color = ('rgb(0, 0, 0)'), shape = 'spline', width = 1))
trace_XV_bounded = go.Scatter(x = model.constr_bounded.Xv.flatten(), y = np.zeros(model.constr_bounded.Xv.shape[0]), mode = 'markers', name = 'Xv - boundedness', marker = dict(symbol = 'line-ns-open', color = ('rgb(0, 0, 0)')))
fig_bounded = add_traces_to_fig(fig_bounded, [trace_UB, trace_LB, trace_XV_bounded])

pltlyoff.iplot(fig_bounded, filename='')

..Running calculation of K_w ... SKIP - (cached)
..Running calculation of Cholesky factor for K_w ... SKIP - (cached)
..Running preparation step 1 - dependence on (XS, X) ... DONE - time: 0.006 seconds
..Running preparation step 2 - dependence on (XV, X) ... DONE - time: 0.000 seconds
..Running preparation step 3 - dependence on (XS, XV, X) ... DONE - time: 0.002 seconds
..sampling 10000 times from truncated constraint distribution C~|C, Y DONE - time: 0.134 seconds
..sampling 10000 times from constrained GP f*|C, Y DONE - time: 0.543 seconds
..computing statistics from samples DONE - time: 0.140 seconds
 DONE - Total time: 0.827 seconds


In [11]:
# Reset model (clear cache)
model.reset()

constr_bounded.Xv = None
model.constr_bounded = constr_bounded

constr_deriv.Xv = None
model.constr_deriv = [constr_deriv] # Add list of constraints for multi-dimensional functions

print(model)

----- GP model ----- 
 mean = 0 
 likelihood = 1e-06 
 kernel: 
   type = RBF 
   input dim = 1 
   lenghtscale = [0.1] 
   variance = 0.5 
 constraint: 
   f [0], df/dx_1 [0] 
   constr_likelihood = 1e-06 
---------------------


In [12]:
# Search for a suitable set of virtual observation locations where the constraint is imposed
df, num_pts, pc_min = model.find_XV_subop(bounds = [(0.001, 1)], p_target = 0.9, i_range = [0, 1], print_intermediate = True)

Searching for points XV s.t. P(a - nu < Lf < b + nu) > p_target = 0.9 for Lf = [f, df/dx_1] and nu = 1.2815515655446004e-06 ...
i = 1, XV[1] = [0.70582493], prob = 0.1642911779065186, optimization time = 0.057 seconds
i = 1, XV[2] = [0.86585777], prob = 0.020021721994449824, optimization time = 0.111 seconds
i = 1, XV[3] = [1.], prob = 0.1174266109752433, optimization time = 0.130 seconds
i = 0, XV[4] = [1.], prob = 0.2205296705068193, optimization time = 0.139 seconds
i = 1, XV[5] = [0.61351164], prob = 0.2170876758863302, optimization time = 0.148 seconds
i = 1, XV[6] = [0.76597563], prob = 0.042833524310164066, optimization time = 0.121 seconds
i = 1, XV[7] = [0.94452481], prob = 0.02779114662943468, optimization time = 0.080 seconds
i = 1, XV[8] = [0.6530911], prob = 0.03785276162869959, optimization time = 0.140 seconds
i = 1, XV[9] = [0.81088761], prob = 0.3250053323150157, optimization time = 0.133 seconds
i = 1, XV[10] = [0.90749087], prob = 0.4298502101438326, optimization tim

In [13]:
mean, var, perc, mode, samples, times = model.calc_posterior_constrained(x_test.reshape(-1, 1), compute_mode = False, num_samples = 1000, save_samples = 5, algorithm = 'minimax_tilting', resample = False)

mean = np.array(mean).flatten()
p_lower = perc[0]
median = perc[1]
p_upper = perc[2]
p_label = '[p{}, p{}]'.format(10, 90)

samplepaths_Z = np.array(samples)

fig_both = PlotGP2d(x_mean = x_test, mean = mean,
                        x_obs = model.X_training[:,0], y_obs = model.Y_training, 
                        p_lower = p_lower, p_upper = p_upper, p_label = p_label,
                        samplepaths =  samplepaths_Z,
                        x_true = x_test, y_true = y_true,
                        title = 'Both constraints', xrange = [0, 1], yrange = [-1.7, 1.7], smoothing = True)

trace_XV_bounded = go.Scatter(x = model.constr_bounded.Xv.flatten(), y = np.zeros(model.constr_bounded.Xv.shape[0]), mode = 'markers', name = 'Xv - boundedness', marker = dict(symbol = 'line-ns-open', color = ('rgb(0, 0, 0)')))
trace_XV_mon = go.Scatter(x = model.constr_deriv[0].Xv.flatten(), y = np.zeros(model.constr_deriv[0].Xv.shape[0]), mode = 'markers', name = 'Xv - monotonicity', marker = dict(symbol = 'x-thin-open', color = ('rgb(0, 0, 0)')))

fig_both = add_traces_to_fig(fig_both, [trace_UB, trace_LB, trace_XV_bounded, trace_XV_mon])

pltlyoff.iplot(fig_both, filename='')

..Running calculation of K_w ... SKIP - (cached)
..Running calculation of Cholesky factor for K_w ... SKIP - (cached)
..Running preparation step 1 - dependence on (XS, X) ... DONE - time: 0.010 seconds
..Running preparation step 2 - dependence on (XV, X) ... SKIP - (cached)
..Running preparation step 3 - dependence on (XS, XV, X) ... DONE - time: 0.002 seconds
..using old samples from truncated constraint distribution C~|C, Y
..sampling 1000 times from constrained GP f*|C, Y DONE - time: 0.388 seconds
..computing statistics from samples DONE - time: 0.012 seconds
 DONE - Total time: 0.413 seconds
